In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [105]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract  

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")


In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
#Starting from the last data point in the database. 
prev_year = dt.date(2017, 8, 23)
# Calculate the date one year from the last date in data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
# Perform a query to retrieve the data and precipitation scores
results = []
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date','precipitation'])
df.set_index(df['date'], inplace=True)

# Sort the dataframe by date
df = df.sort_index()
# Use Pandas Plotting with Matplotlib to plot the data
df.plot()
plt.xticks(rotation=90)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.station)). \
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
session.query(Measurement.tobs)
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    filter(Measurement.date >= prev_year).all()

df = pd.DataFrame(results, columns=['tobs'])

df.plot.hist(bins=12)
plt.tight_layout()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates

start = dt.datetime(2017,6,1)
end = dt.datetime(2017,6,30)
june = [start + dt.timedelta(days=x) for x in range(0, (end-start).days)]

session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(june).all()


# Challenge

In [122]:
#get all data from June 2010-2017
june_data = session.query(Measurement.date, Measurement.prcp,
                         Measurement.tobs, Station.station).filter(extract('month',Measurement.date)==6).all()
june_data_df=pd.DataFrame(june_data)
june_data_df.rename(columns={'date': 'Date', 'prcp': 'Precipitation', 'tobs' : 'Temp', 'station' : 'Station'}, inplace=True)
june_data_df

,Date,Precipitation,Temp,Station
0,2010-06-01,0.0,78.0,USC00519397
1,2010-06-01,0.0,78.0,USC00513117
2,2010-06-01,0.0,78.0,USC00514830
3,2010-06-01,0.0,78.0,USC00517948
4,2010-06-01,0.0,78.0,USC00518838
...,...,...,...,...
15295,2017-06-30,0.2,75.0,USC00518838
15296,2017-06-30,0.2,75.0,USC00519523
15297,2017-06-30,0.2,75.0,USC00519281
15298,2017-06-30,0.2,75.0,USC00511918


In [123]:
#get statistical data from June 2010-2017
june_data_df.describe()

,Precipitation,Temp
count,14166.000000,15300.000000
mean,0.136360,74.944118
std,0.335636,3.256565
min,0.000000,64.000000
25%,0.000000,73.000000
50%,0.020000,75.000000
75%,0.120000,77.000000
max,4.430000,85.000000


In [121]:
#get all data from December 2010-2017
dec_data = session.query(Measurement.date, Measurement.prcp,
                         Measurement.tobs, Station.station).filter(extract('month',Measurement.date)==1).all()
dec_data_df=pd.DataFrame(dec_data)
dec_data_df.rename(columns={'date': 'Date', 'prcp': 'Precipitation', 'tobs' : 'Temp', 'station' : 'Station'}, inplace=True)
dec_data_df

,Date,Precipitation,Temp,Station
0,2010-01-01,0.08,65.0,USC00519397
1,2010-01-01,0.08,65.0,USC00513117
2,2010-01-01,0.08,65.0,USC00514830
3,2010-01-01,0.08,65.0,USC00517948
4,2010-01-01,0.08,65.0,USC00518838
...,...,...,...,...
15538,2017-01-31,0.00,65.0,USC00518838
15539,2017-01-31,0.00,65.0,USC00519523
15540,2017-01-31,0.00,65.0,USC00519281
15541,2017-01-31,0.00,65.0,USC00511918


In [126]:
#get statistucal data from December 2010-2017
dec_data_df.describe()


,Precipitation,Temp
count,14580.000000,15543.000000
mean,0.129975,68.726115
std,0.477931,4.366767
min,0.000000,54.000000
25%,0.000000,66.000000
50%,0.000000,69.000000
75%,0.050000,72.000000
max,8.810000,81.000000
